In [1]:
import numpy as np
from queue import PriorityQueue

In [2]:
class GraphDomain:
    def __init__(self, path, name = 'NULL', pad = 1):
        self.NAME = name
        self.NUM_NODE = 0
        self.NUM_DOMAIN = 0
        self.START_NODE = -1
        self.END_NODE = -1

        self.adj = {}
        self.distance = {}
        self.pre_node = {}
        self.domain_start_nodes = {}
        
        self.pad = pad
        self.load_data(path)
        self.build_graph()

        self.best = np.inf
 
        
    def load_data(self, path):
        f = open(path, 'r')

        l1 = f.readline()
        l1 = l1.strip()
        l1 = l1.split(' ')
        self.NUM_NODE = int(l1[0])
        self.NUM_DOMAIN = int(l1[1])

        l2 = f.readline()
        l2 = l2.strip()
        l2 = l2.split(' ')
        self.START_NODE = int(l2[0])
        self.END_NODE = int(l2[1])

        # fomat edge: start, end, weight, domain
        edges = f.readlines()

        f.close()

        #setup
        for d in range(self.pad, self.pad+self.NUM_DOMAIN):
            self.adj[d] = {}
            self.distance[d] = {}
            self.domain_start_nodes[d] = []
            self.pre_node[d] = {}
            for v in range(self.pad, self.pad+self.NUM_NODE):
                self.adj[d][v] = list()
                self.distance[d][v] = np.full((self.NUM_NODE+self.pad,), np.inf)
                self.pre_node[d][v] = np.full((self.NUM_NODE+self.pad,), -1)
        
        for e in edges:
            e = e.strip()
            e = e.split(' ')
            #u = e[0], v = e[1], w = e[2], d = e[3]
            self.adj[int(e[3])][int(e[0])].append((int(e[1]), int(e[2])))
            
        

    def Dijkstra(self, domain, start, to_show=False):
        if(to_show):
            dis = np.full((self.NUM_NODE+self.pad,), np.inf)
        else:
            dis = self.distance[domain][start]
        # pre = np.full((self.NUM_NODE+self.pad,), -1)
        pre = self.pre_node[domain][start]
        dis[start] = 0
        pre[start] = start

        PQ = PriorityQueue()
        # PQ.put((dis[start], start))
        PQ.put((0, start))

        while not PQ.empty():
            curr = PQ.get()
            u = curr[1]
            d = curr[0]

            for e in self.adj[domain][u]:
                if e[1] + d < dis[e[0]]:
                    dis[e[0]] = e[1] + d
                    pre[e[0]] = u
                    PQ.put((dis[e[0]], e[0]))
        
        # return pre
    

    def build_graph(self):
        for d in range(self.pad, self.NUM_DOMAIN+self.pad):
            for v in range(self.pad, self.NUM_NODE+self.pad):
                self.Dijkstra(d, v)
                if len(self.adj[d][v]) != 0:
                    self.domain_start_nodes[d].append(v)

In [3]:
import os

sc = "D:/MachineLearning/EvolutionComputation/IDPCDU/Datasets/IDPCDU_Edges/set1/"
TaskNames = os.listdir(sc)
TaskNames.sort()
for i in range(len(TaskNames)):
    print(i, TaskNames[i])

0 idpc_10x10x1000.idpc
1 idpc_10x20x2713.idpc
2 idpc_10x5x425.idpc
3 idpc_15x15x3375.idpc
4 idpc_15x30x12111.idpc
5 idpc_15x7x1504.idpc
6 idpc_20x10x2492.idpc
7 idpc_20x20x8000.idpc
8 idpc_20x40x26104.idpc
9 idpc_25x12x4817.idpc
10 idpc_25x25x15625.idpc
11 idpc_25x50x57147.idpc
12 idpc_30x15x10025.idpc
13 idpc_30x30x27000.idpc
14 idpc_30x60x89772.idpc
15 idpc_35x17x13934.idpc
16 idpc_35x35x42875.idpc
17 idpc_35x70x123585.idpc
18 idpc_40x20x18485.idpc
19 idpc_40x40x64000.idpc
20 idpc_40x80x130681.idpc
21 idpc_45x22x43769.idpc
22 idpc_45x45x91125.idpc
23 idpc_45x90x322081.idpc


In [4]:
test = GraphDomain(sc+TaskNames[0])

In [5]:
test.START_NODE

1

In [6]:
test.END_NODE

10

In [7]:
adj = [[] for i in range(test.NUM_NODE+1)]
mark_domain = [np.full((test.NUM_NODE+1,), -1) for i in range(test.NUM_NODE+1)]

In [8]:

for i in range(1, test.NUM_NODE+1):
    for j in range(1, test.NUM_NODE+1):
        dis = np.inf
        domain = -1
        for d in range(1, test.NUM_DOMAIN+1):
            if dis > test.distance[d][i][j]:
                dis = test.distance[d][i][j]
                domain = d
        adj[i].append((j, dis))
        mark_domain[i][j] = domain

In [31]:
dis = np.full((test.NUM_NODE+1,), np.inf)
pre = np.full((test.NUM_NODE+1,), -1)
dis[test.START_NODE] = 0
pre[test.START_NODE] = test.START_NODE

PQ = PriorityQueue()
PQ.put((0, test.START_NODE))

while not PQ.empty():
    curr = PQ.get()
    u = curr[1]
    d = curr[0]

    for e in adj[u]:
        if e[1] + d < dis[e[0]]:
            dis[e[0]] = e[1] + d
            pre[e[0]] = u
            PQ.put((dis[e[0]], e[0]))

In [10]:
dis[test.END_NODE]

7.0

In [32]:
path = []
path.append(test.END_NODE)
curr = test.END_NODE

while curr != pre[curr]:
    curr = pre[curr]
    path.append(curr)

path = path[::-1]

In [12]:
domain = []
for i in range(len(path)-1):
    domain.append(mark_domain[path[i]][path[i+1]])

In [13]:
if len(domain) == len(set(domain)):
    print('TRUE')

TRUE


In [14]:
print(len(path))
print(path)

7
[1, 6, 9, 2, 8, 7, 10]


In [15]:
print(len(domain))
print(domain)

6
[2, 6, 8, 3, 9, 4]


In [33]:
for v in range(len(path)-1):
    temp = []

    for i in range(1, test.NUM_DOMAIN+1):
        temp.append(test.distance[i][path[v]][7])

    print(np.sort(np.array(temp)))

[13. 14. 14. 15. 16. 17. 18. 22. 25. 26.]
[11. 12. 12. 15. 15. 16. 18. 18. 18. 23.]
[11. 14. 16. 19. 22. 26. 27. 29. 32. 34.]
[12. 14. 16. 17. 17. 18. 18. 19. 19. 27.]
[ 1. 11. 13. 14. 21. 27. 30. 30. 34. 39.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
from build import GraphDomain as gd

t = gd(sc+TaskNames[0])

In [26]:
t.Cost([2, 6, 8, 3, 9, 4])

(7.0, 7)

In [35]:
dis = np.full((test.NUM_NODE+1,), np.inf)
pre = np.full((test.NUM_NODE+1,), -1)
dis[4] = 0
pre[4] = 4

PQ = PriorityQueue()
PQ.put((0, 4))

while not PQ.empty():
    curr = PQ.get()
    u = curr[1]
    d = curr[0]

    for e in adj[u]:
        if e[1] + d < dis[e[0]]:
            dis[e[0]] = e[1] + d
            pre[e[0]] = u
            PQ.put((dis[e[0]], e[0]))

In [37]:
dis

array([inf, 11., 12., 12.,  0., 11., 11., 12., 12., 11., 13.])

In [36]:
for i in range(1, 11):
    path = []
    path.append(i)
    curr = i

    while curr != pre[curr]:
        curr = pre[curr]
        path.append(curr)

    path = path[::-1]

    domain = []
    for i in range(len(path)-1):
        domain.append(mark_domain[path[i]][path[i+1]])

    print(domain)

[3]
[1]
[8]
[]
[1]
[8]
[9]
[4]
[8]
[1]


#### **Run**

In [ ]:
import os

sc = "D:/MachineLearning/EvolutionComputation/IDPCDU/Datasets/IDPCDU_Edges/set1/"
TaskNames = os.listdir(sc)
TaskNames.sort()
for i in range(len(TaskNames)):
    print(i, TaskNames[i])

In [85]:
for t in range(24):
    test = GraphDomain(sc+TaskNames[t])

    adj = [[] for i in range(test.NUM_NODE+1)]
    mark_domain = [np.full((test.NUM_NODE+1,), -1) for i in range(test.NUM_NODE+1)]

    for i in range(1, test.NUM_NODE+1):
        for j in range(1, test.NUM_NODE+1):
            dis = np.inf
            domain = -1
            for d in range(1, test.NUM_DOMAIN+1):
                if dis > test.distance[d][i][j]:
                    dis = test.distance[d][i][j]
                    domain = d
            adj[i].append((j, dis))
            mark_domain[i][j] = domain

    dis = np.full((test.NUM_NODE+1,), np.inf)
    pre = np.full((test.NUM_NODE+1,), -1)
    dis[test.START_NODE] = 0
    pre[test.START_NODE] = test.START_NODE

    PQ = PriorityQueue()
    PQ.put((0, test.START_NODE))

    while not PQ.empty():
        curr = PQ.get()
        u = curr[1]
        d = curr[0]

        for e in adj[u]:
            if e[1] + d < dis[e[0]]:
                dis[e[0]] = e[1] + d
                pre[e[0]] = u
                PQ.put((dis[e[0]], e[0]))

    path = []
    path.append(test.END_NODE)
    curr = test.END_NODE

    while curr != pre[curr]:
        curr = pre[curr]
        path.append(curr)

    path = path[::-1]

    domain = []
    for i in range(len(path)-1):
        domain.append(mark_domain[path[i]][path[i+1]])

    if len(domain) == len(set(domain)):
        print(t, " ", TaskNames[t], "  \t", dis[test.END_NODE])

0   idpc_10x10x1000.idpc   	 7.0
1   idpc_10x20x2713.idpc   	 7.0
2   idpc_10x5x425.idpc   	 7.0
3   idpc_15x15x3375.idpc   	 10.0
4   idpc_15x30x12111.idpc   	 10.0
5   idpc_15x7x1504.idpc   	 10.0
6   idpc_20x10x2492.idpc   	 15.0
7   idpc_20x20x8000.idpc   	 15.0
8   idpc_20x40x26104.idpc   	 15.0
9   idpc_25x12x4817.idpc   	 18.0
10   idpc_25x25x15625.idpc   	 18.0
11   idpc_25x50x57147.idpc   	 18.0
12   idpc_30x15x10025.idpc   	 24.0
13   idpc_30x30x27000.idpc   	 24.0
14   idpc_30x60x89772.idpc   	 24.0
15   idpc_35x17x13934.idpc   	 28.0
16   idpc_35x35x42875.idpc   	 28.0
17   idpc_35x70x123585.idpc   	 28.0
18   idpc_40x20x18485.idpc   	 32.0
19   idpc_40x40x64000.idpc   	 32.0
20   idpc_40x80x130681.idpc   	 32.0
21   idpc_45x22x43769.idpc   	 35.0
22   idpc_45x45x91125.idpc   	 35.0
23   idpc_45x90x322081.idpc   	 35.0


In [87]:
import os

sc = "D:/MachineLearning/EvolutionComputation/IDPCDU/Datasets/IDPCDU_Edges/set2/"
TaskNames = os.listdir(sc)
TaskNames.sort()
for i in range(len(TaskNames)):
    print(i, TaskNames[i])

0 idpc_100x100x1000000.idpc
1 idpc_100x200x2296097.idpc
2 idpc_100x50x461319.idpc
3 idpc_50x100x285357.idpc
4 idpc_50x25x38961.idpc
5 idpc_50x50x125000.idpc
6 idpc_60x120x434337.idpc
7 idpc_60x30x99470.idpc
8 idpc_60x60x216000.idpc
9 idpc_70x140x923343.idpc
10 idpc_70x35x120810.idpc
11 idpc_70x70x343000.idpc
12 idpc_80x160x1490468.idpc
13 idpc_80x40x175762.idpc
14 idpc_80x80x512000.idpc
15 idpc_90x180x1644367.idpc
16 idpc_90x45x260195.idpc
17 idpc_90x90x729000.idpc


In [88]:
for t in range(18):
    test = GraphDomain(sc+TaskNames[t])

    adj = [[] for i in range(test.NUM_NODE+1)]
    mark_domain = [np.full((test.NUM_NODE+1,), -1) for i in range(test.NUM_NODE+1)]

    for i in range(1, test.NUM_NODE+1):
        for j in range(1, test.NUM_NODE+1):
            dis = np.inf
            domain = -1
            for d in range(1, test.NUM_DOMAIN+1):
                if dis > test.distance[d][i][j]:
                    dis = test.distance[d][i][j]
                    domain = d
            adj[i].append((j, dis))
            mark_domain[i][j] = domain

    dis = np.full((test.NUM_NODE+1,), np.inf)
    pre = np.full((test.NUM_NODE+1,), -1)
    dis[test.START_NODE] = 0
    pre[test.START_NODE] = test.START_NODE

    PQ = PriorityQueue()
    PQ.put((0, test.START_NODE))

    while not PQ.empty():
        curr = PQ.get()
        u = curr[1]
        d = curr[0]

        for e in adj[u]:
            if e[1] + d < dis[e[0]]:
                dis[e[0]] = e[1] + d
                pre[e[0]] = u
                PQ.put((dis[e[0]], e[0]))

    path = []
    path.append(test.END_NODE)
    curr = test.END_NODE

    while curr != pre[curr]:
        curr = pre[curr]
        path.append(curr)

    path = path[::-1]

    domain = []
    for i in range(len(path)-1):
        domain.append(mark_domain[path[i]][path[i+1]])

    if len(domain) == len(set(domain)):
        print(t, " ", TaskNames[t], "  \t", dis[test.END_NODE])

0   idpc_100x100x1000000.idpc   	 80.0
1   idpc_100x200x2296097.idpc   	 80.0
2   idpc_100x50x461319.idpc   	 80.0
3   idpc_50x100x285357.idpc   	 38.0
4   idpc_50x25x38961.idpc   	 38.0
5   idpc_50x50x125000.idpc   	 38.0
6   idpc_60x120x434337.idpc   	 45.0
7   idpc_60x30x99470.idpc   	 45.0
8   idpc_60x60x216000.idpc   	 45.0
9   idpc_70x140x923343.idpc   	 55.0
10   idpc_70x35x120810.idpc   	 55.0
11   idpc_70x70x343000.idpc   	 55.0
12   idpc_80x160x1490468.idpc   	 70.0
13   idpc_80x40x175762.idpc   	 70.0
14   idpc_80x80x512000.idpc   	 70.0
15   idpc_90x180x1644367.idpc   	 75.0
16   idpc_90x45x260195.idpc   	 75.0
17   idpc_90x90x729000.idpc   	 75.0
